In [1]:
import pandas as pd
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    
dfs = pd.read_csv("./data/final.csv")

In [2]:
df1 =dfs

In [3]:
df1 = df1.dropna()

In [4]:
df = df1.groupby('tag').agg({"pattern": lambda x:list(x),
                       "response" : lambda x: list(x)
                       })

In [5]:
df = df.reset_index(level=0)

In [6]:
corpus = df.to_dict("records")

In [7]:
data = {}
data['intents'] = corpus

In [8]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /Users/yifanzhu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yifanzhu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["pattern"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [10]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [11]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               68992     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 27)                1755      
                                                                 
Total params: 79,003
Trainable params: 79,003
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200


2022-12-03 22:33:01.569531: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


22/22 [==============================] - 0s 845us/step - loss: 3.1445 - accuracy: 0.1202
Epoch 2/200
22/22 [==============================] - 0s 869us/step - loss: 2.3050 - accuracy: 0.3591
Epoch 3/200
22/22 [==============================] - 0s 933us/step - loss: 1.6890 - accuracy: 0.4866
Epoch 4/200
22/22 [==============================] - 0s 828us/step - loss: 1.2847 - accuracy: 0.6157
Epoch 5/200
22/22 [==============================] - 0s 993us/step - loss: 1.0285 - accuracy: 0.7003
Epoch 6/200
22/22 [==============================] - 0s 883us/step - loss: 0.7903 - accuracy: 0.7596
Epoch 7/200
22/22 [==============================] - 0s 877us/step - loss: 0.6541 - accuracy: 0.8012
Epoch 8/200
22/22 [==============================] - 0s 970us/step - loss: 0.6099 - accuracy: 0.7908
Epoch 9/200
22/22 [==============================] - 0s 979us/step - loss: 0.4347 - accuracy: 0.8665
Epoch 10/200
22/22 [==============================] - 0s 1ms/step - loss: 0.5374 - accuracy: 0.8442
Epo

In [12]:
def clean_text(text): 
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab): 
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    return np.array(bow)

def pred_class(text, vocab, labels): 
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]),verbose=0)[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list[0]



In [13]:
#Load necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

/Users/yifanzhu/opt/miniconda3/envs/anly-501/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
# Splits each line of csv file to create conversation_object and conversations
def loadConversations(filepath):

    conversations = {}
    with open(filepath) as csvfile:
        reader = csv.DictReader(csvfile)
        id_index = 1
        for row in reader:
            conversation_object = {}
            conversation_object['id'] = id_index
            conversation_object['tag'] = row['tag']
            conversation_object['pattern'] = row['pattern']
            conversation_object['response'] = row['response']

            conversations[conversation_object['id']] = conversation_object

            id_index +=1

    return conversations


# Extracts pairs of pattern and response from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations.values():

        # Iterate over each conversation and retrieve pattern and response
        pattern = conversation["pattern"].strip()
        response = conversation["response"].strip()
        qa_pairs.append([pattern, response])

    return qa_pairs

In [48]:
# running the chatbot
print(color.GREEN+"ROBO: Hello. If you want to exit, type Bye!"+color.END)
while True:
    message = input("Me: ")
    if message.lower() == 'bye':
        print(color.GREEN+"ROBO: Bye! take care.."+color.END)
        break
    
    tag = pred_class(message, words, classes)

    # Define path to new file with formatted data
    datafile = os.path.join('data/formatted_lines2.txt')

    delimiter = '\t'
    # Unescape the delimiter
    delimiter = str(codecs.decode(delimiter, "unicode_escape"))

    # Initialize conversations dict
    conversations = {}
    # Load conversations
    conversations = loadConversations('data/final.csv')

    # Write new file
    with open(datafile, 'w', encoding='utf-8') as outputfile:
        writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
        for pair in extractSentencePairs(conversations):
            writer.writerow(pair)

    # Default word tokens
    PAD_token = 0  # Used for padding short sentences
    SOS_token = 1  # Start-of-sentence token
    EOS_token = 2  # End-of-sentence token

    class Voc:
        def __init__(self):
            self.trimmed = False
            self.word2index = {}
            self.word2count = {}
            self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
            self.num_words = 3  # Count SOS, EOS, PAD

        def addSentence(self, sentence):
            for word in sentence.split(' '):
                self.addWord(word)

        def addWord(self, word):
            if word not in self.word2index:
                self.word2index[word] = self.num_words
                self.word2count[word] = 1
                self.index2word[self.num_words] = word
                self.num_words += 1
            else:
                self.word2count[word] += 1

        # Remove words below a certain count threshold
        def trim(self, min_count):
            if self.trimmed:
                return
            self.trimmed = True

            keep_words = []

            for k, v in self.word2count.items():
                if v >= min_count:
                    keep_words.append(k)

            print('keep_words {} / {} = {:.4f}'.format(
                len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
            ))

            # Reinitialize dictionaries
            self.word2index = {}
            self.word2count = {}
            self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
            self.num_words = 3 # Count default tokens

            for word in keep_words:
                self.addWord(word)

    MAX_LENGTH = 20  # Maximum sentence length to consider

    # Turn a Unicode string to plain ASCII, thanks to
    # https://stackoverflow.com/a/518232/2809427
    def unicodeToAscii(s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )

    # Lowercase, trim, and remove non-letter characters
    def normalizeString(s):
        s = unicodeToAscii(s.lower().strip())
        s = re.sub(r"([.!?])", r" \1", s)
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
        s = re.sub(r"\s+", r" ", s).strip()
        return s

    # Read pattern/response pairs and return a voc object
    def readVocs(datafile):
        # Read the file and split into pairs
        lines = open(datafile, encoding='utf-8').\
            read().strip().split('\n')
        # Split every line into pairs and normalize
        pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
        voc = Voc()
        return voc, pairs

    # Returns True if both sentences in a pair 'p' are under the MAX_LENGTH threshold
    def filterPair(p):
        # Input sequences need to preserve the last word for EOS token
        return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

    # Filter pairs using filterPair condition
    def filterPairs(pairs):
        return [pair for pair in pairs if filterPair(pair)]

    taglist = ['start','greeting','goodbye','thanks','no-response','help','sad','stressed','worthless','depressed','happy',
    'anxious','not-talking','sleep','scared','death','understand','done','suicide','default','wrong','ask','user-advice',
    'meditation','learn-mental-health','Angry','User-agree','learn-more','no-approach']

    # Using the functions defined above, return a populated voc object and pairs list
    def loadPrepareData(datafile,tag):
        voc, pairs = readVocs(datafile)
        print("Tag identified as:", tag)
        pairs = pairs[25*(taglist.index(tag)-1):25*taglist.index(tag)]
        print("Read {!s} sentence pairs".format(len(pairs)))
        pairs = filterPairs(pairs)
        print("Trimmed to {!s} sentence pairs".format(len(pairs)))
        for pair in pairs:
            voc.addSentence(pair[0])
            voc.addSentence(pair[1])
        print("Counted words:", voc.num_words)
        return voc, pairs

    # voc, pairs = loadPrepareData(datafile,tag)
    # Print some pairs to validate
    # print("\npairs:")
    # for pair in pairs:
    #     print(pair)

      
    # Load/Assemble voc and pairs
    # save_dir = os.path.join("data", "save")
    voc, pairs = loadPrepareData(datafile,tag)

    def indexesFromSentence(voc, sentence):
        return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


    def zeroPadding(l, fillvalue=PAD_token):
        return list(itertools.zip_longest(*l, fillvalue=fillvalue))

    def binaryMatrix(l, value=PAD_token):
        m = []
        for i, seq in enumerate(l):
            m.append([])
            for token in seq:
                if token == PAD_token:
                    m[i].append(0)
                else:
                    m[i].append(1)
        return m

    # Returns padded input sequence tensor and lengths
    def inputVar(l, voc):
        indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
        lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
        padList = zeroPadding(indexes_batch)
        padVar = torch.LongTensor(padList)
        return padVar, lengths

    # Returns padded target sequence tensor, padding mask, and max target length
    def outputVar(l, voc):
        indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
        max_target_len = max([len(indexes) for indexes in indexes_batch])
        padList = zeroPadding(indexes_batch)
        mask = binaryMatrix(padList)
        mask = torch.BoolTensor(mask)
        padVar = torch.LongTensor(padList)
        return padVar, mask, max_target_len

    # Returns all items for a given batch of pairs
    def batch2TrainData(voc, pair_batch):
        pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
        input_batch, output_batch = [], []
        for pair in pair_batch:
            input_batch.append(pair[0])
            output_batch.append(pair[1])
        inp, lengths = inputVar(input_batch, voc)
        output, mask, max_target_len = outputVar(output_batch, voc)
        return inp, lengths, output, mask, max_target_len


    # Example for validation
    small_batch_size = 5
    batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
    input_variable, lengths, target_variable, mask, max_target_len = batches

    # print("input_variable:", input_variable)
    # print("lengths:", lengths)
    # print("target_variable:", target_variable)
    # print("mask:", mask)
    # print("max_target_len:", max_target_len)


    ###Reference from Pytorch
    class EncoderRNN(nn.Module):
        def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
            super(EncoderRNN, self).__init__()
            self.n_layers = n_layers
            self.hidden_size = hidden_size
            self.embedding = embedding

            # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
            # because our input size is a word embedding with number of features == hidden_size
            self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                            dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

        def forward(self, input_seq, input_lengths, hidden=None):
            # Convert word indexes to embeddings (max_length,batch) -> (max_length,batch,hidden_size)
            embedded = self.embedding(input_seq)
            # Pack padded batch of sequences for RNN module
            packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
            # Forward pass through GRU
            # Tensor(max_length, batch, hidden_size) 
            # Output.view(seq_len, batch, num_directions, hidden_size)
            outputs, hidden = self.gru(packed, hidden)
            # Unpack output calculations
            outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
            # Sum bidirectional GRU outputs 
            outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
            # Return output and final hidden state
            return outputs, hidden
    
    # Luong attention layer
    class Attn(nn.Module):
        def __init__(self, hidden_size):
            super(Attn, self).__init__()

        #Use dot product of h_t and h_s to calculate attention scores
        def dot_score(self, hidden, encoder_output):
            return torch.sum(hidden * encoder_output, dim=2)

        def forward(self, hidden, encoder_outputs):
            attn_energies = self.dot_score(hidden, encoder_outputs)

            # Transpose max_length and batch_size dimensions
            attn_energies = attn_energies.t()

            # Return the softmax normalized probability scores (with added dimension)
            # Use unsequeeze(1) -> (64,1,10)
            return F.softmax(attn_energies, dim=1).unsqueeze(1)
    

    ###Reference from Pytorch
    class LuongAttnDecoderRNN(nn.Module):
        def __init__(self, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
            super(LuongAttnDecoderRNN, self).__init__()

            # Keep for reference
            self.hidden_size = hidden_size
            self.output_size = output_size
            self.n_layers = n_layers
            self.dropout = dropout

            # Define layers
            self.embedding = embedding
            self.embedding_dropout = nn.Dropout(dropout)
            self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
            self.concat = nn.Linear(hidden_size * 2, hidden_size)
            self.out = nn.Linear(hidden_size, output_size)

            self.attn = Attn(hidden_size)

        def forward(self, input_step, last_hidden, encoder_outputs):
            # we process data one at a time t -> t+1
            # Get embedding of current input word
            embedded = self.embedding(input_step)
            embedded = self.embedding_dropout(embedded)
            # Forward through unidirectional GRU
            rnn_output, hidden = self.gru(embedded, last_hidden)
            # Calculate attention weights from the current GRU output
            attn_weights = self.attn(rnn_output, encoder_outputs)
            # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
            # bmm -> 64 (1,10) X (10, 500) -> (64,1,500)
            context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
            # Concatenate weighted context vector and GRU output using Luong eq. 5
            rnn_output = rnn_output.squeeze(0)
            context = context.squeeze(1)
            concat_input = torch.cat((rnn_output, context), 1)
            concat_output = torch.tanh(self.concat(concat_input))
            # Predict next word using Luong eq. 6
            output = self.out(concat_output)
            output = F.softmax(output, dim=1)
            # Return output and final hidden state
            return output, hidden
    

    ###Reference from Pytorch
    def maskNLLLoss(inp, target, mask):
        nTotal = mask.sum()
        crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
        loss = crossEntropy.masked_select(mask).mean()
        loss = loss.to(device)
        return loss, nTotal.item()

    def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):
    
        #emptry gradiant
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        #initialize variables
        loss = 0
        print_losses = []
        n_totals = 0

        #forward calculation for encoder
        encoder_output, encoder_hidden = encoder(input_variable, lengths)

        #create decoder input
        decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
        #decoder_input = decoder_input.to(device)

        #initialize decoder hidden state to final encoder hidden state
        decoder_hidden = encoder_hidden[:decoder.n_layers]

        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input,decoder_hidden,encoder_output)

            #next decoder_input is current target which the decoder_input is always correct
            decoder_input = target_variable[t].view(1, -1)

            #calculate accumulated loss
            mask_loss, nTotal = maskNLLLoss(decoder_output,target_variable[t],mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

        #backpropatation
        loss.backward()

        #perform gradiant clipping
        _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
        _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

        #update parameters
        encoder_optimizer.step()
        decoder_optimizer.step()

        return sum(print_losses) / n_totals
    


    def trainIters(voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, n_iteration, batch_size, print_every, clip, loadFilename):
    
        #Randomly pick number of iteration batch
        training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
        for _ in range(n_iteration)]

        #Initialization
        # print('Initializing ...')
        start_iteration = 1
        print_loss = 0
        
        #Train 
        # print('Training ...')
        for iteration in range(start_iteration, n_iteration + 1):
            training_batch = training_batches[iteration - 1]

            input_variable, lengths, target_variable, mask, max_target_len = training_batch

            #train one batch
            loss = train(input_variable, lengths, target_variable, mask, max_target_len,encoder,
            decoder,embedding,encoder_optimizer,decoder_optimizer,batch_size,clip)
            print_loss += loss

            # Print progress
            if iteration % print_every == 0:
                print_loss_avg = print_loss / print_every
                # print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
                print_loss = 0
    

    ###Reference from Pytorch
    class GreedySearchDecoder(nn.Module):
        def __init__(self, encoder, decoder):
            super(GreedySearchDecoder, self).__init__()
            self.encoder = encoder
            self.decoder = decoder

        def forward(self, input_seq, input_length, max_length):
            # Forward input through encoder model
            encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
            # Prepare encoder's final hidden layer to be first hidden input to the decoder
            decoder_hidden = encoder_hidden[:decoder.n_layers]
            # Initialize decoder input with SOS_token
            decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
            # Initialize tensors to append decoded words to
            all_tokens = torch.zeros([0], device=device, dtype=torch.long)
            all_scores = torch.zeros([0], device=device)
            # Iteratively decode one word token at a time
            for _ in range(max_length):
                # Forward pass through decoder
                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
                # Obtain most likely word token and its softmax score
                decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
                # Record token and score
                all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
                all_scores = torch.cat((all_scores, decoder_scores), dim=0)
                # Prepare current token to be next decoder input (add a dimension)
                decoder_input = torch.unsqueeze(decoder_input, 0)
            # Return collections of word tokens and scores
            return all_tokens, all_scores


    ###Reference from Pytorch
    def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
        ### Format input sentence as a batch
        # words -> indexes
        indexes_batch = [indexesFromSentence(voc, sentence)]
        # Create lengths tensor
        lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
        # Transpose dimensions of batch to match models' expectations
        input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
        # Use appropriate device
        input_batch = input_batch.to(device)
        lengths = lengths.to("cpu")
        # Decode sentence with searcher
        tokens, scores = searcher(input_batch, lengths, max_length)
        # indexes -> words
        decoded_words = [voc.index2word[token.item()] for token in tokens]
        return decoded_words


    def evaluateInput(encoder, decoder, searcher, voc, message):
        # Normalize sentence
        input_sentence = normalizeString(message)
        # Evaluate sentence
        output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
        # Format and print response sentence
        output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
        print('User:', input_sentence)
        print('Bot:', ' '.join(output_words))

    # Configure models
    hidden_size = 500
    encoder_n_layers = 2
    decoder_n_layers = 2
    dropout = 0.1
    batch_size = 64

    # Set checkpoint to load from; set to None if starting from scratch
    loadFilename = None
    checkpoint_iter = 4000

    # Load model if a loadFilename is provided
    if loadFilename:
        checkpoint = torch.load(loadFilename)
        encoder_sd = checkpoint['en']
        decoder_sd = checkpoint['de']
        encoder_optimizer_sd = checkpoint['en_opt']
        decoder_optimizer_sd = checkpoint['de_opt']
        embedding_sd = checkpoint['embedding']
        voc.__dict__ = checkpoint['voc_dict']


    # print('Building encoder and decoder ...')
    # Initialize word embeddings
    embedding = nn.Embedding(voc.num_words, hidden_size)
    if loadFilename:
        embedding.load_state_dict(embedding_sd)
    # Initialize encoder & decoder models
    encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
    decoder = LuongAttnDecoderRNN(embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
    if loadFilename:
        encoder.load_state_dict(encoder_sd)
        decoder.load_state_dict(decoder_sd)
    # Use appropriate device
    encoder = encoder.to(device)
    decoder = decoder.to(device)
    # print('Models built and ready to go!')

    # Configure training/optimization
    clip = 50.0
    learning_rate = 0.0001
    decoder_learning_ratio = 5.0
    n_iteration = 25
    print_every = 1

    # Ensure dropout layers are in train mode
    encoder.train()
    decoder.train()

    # Initialize optimizers
    # print('Building optimizers ...')
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
    if loadFilename:
        encoder_optimizer.load_state_dict(encoder_optimizer_sd)
        decoder_optimizer.load_state_dict(decoder_optimizer_sd)

    # If you have cuda, configure cuda to call
    for state in encoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    for state in decoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    # Run training iterations
    # print("Starting Training!")
    trainIters(voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
            embedding, encoder_n_layers, decoder_n_layers, n_iteration, batch_size,
            print_every, clip, loadFilename)

    
    # Set dropout layers to eval mode
    encoder.eval()
    decoder.eval()

    # Initialize search module
    searcher = GreedySearchDecoder(encoder, decoder)
    # Answer question
    
    evaluateInput(encoder, decoder, searcher, voc, message)

    
    

ROBO: Hello. If you want to exit, type Bye!
Tag identified as: greeting
Read 25 sentence pairs
Trimmed to 24 sentence pairs
Counted words: 78
User: hi
Bot: how are you ? ?
Tag identified as: Angry
Read 25 sentence pairs
Trimmed to 25 sentence pairs
Counted words: 118
User: i m angry
Bot: i m you you ll better . . . you .
Tag identified as: suicide
Read 25 sentence pairs
Trimmed to 23 sentence pairs
Counted words: 139
User: i want to kill myself
Bot: i m you you you t t a way out . .


KeyboardInterrupt: Interrupted by user